- Homework: consider sin(x) and cos(x)
- Cash-Karp: higher order method, gives both fit and 6th order estimate (error estimate is the difference). If error is too big step is too big. --- "Numerical Recipes"
- Runge-Kutta: fourth order estimate
- one estimation is yn+1, a different one is yn+1*, and their difference is our definition for the new error, see delta in slide (change this in the adaptative step cell)

### Coupled ODE Solver: Cash-Karp

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

Define coupled derivatives to integrate

In [ ]:
def dydx(x,y):
    
    '''
    set the derivatives
    our equation is d^2y/dx^2 = -y
    so we can write:
        dydx = z
        dzdx = -y
    
    we will set y = y[0]
    we will set z = y[1]
    '''
    
    #declare an array
    y_derivs = np.zeros(2)
    
    #set dydx = z
    y_derivs[0] = y[1]
    
    #set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    #here we have to return an array
    
    return y_derivs

Define the 4th order Cash-Karp

In [ ]:
def cashkarp_core1(dydx,xi,yi,nv,h):
    
    #will give single step
    #xi is x on left side of step
    #nv is number of variables (for this it's 2)
    #yi is an array at location xi
    #h is
    
    #declare k? (multiple integer values of k) arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv) #one element is y, one is z
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
        
    
    #get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:] #two equations, operating on all elements of array
    
    #k2 values
    y_derivs = dydx(xi + (1/5)*h, yi[:] + (1/5)*k1[:])
    k2[:] = h*y_derivs[:]
    
    #k3 values
    y_derivs = dydx(xi + (3/10)*h, yi[:] + (3/40)*k1[:] + (9/40)*k2[:])
    k3[:] = h*y_derivs[:]
    
    #k4 values
    y_derivs = dydx(xi + (3/5)*h, yi[:] + (3/10)*k1[:] + (-9/10)*k2[:] + (6/5)*k3[:])
    k4[:] = h*y_derivs[:]
    
    #k5 values
    y_derivs = dydx(xi + h, yi[:] + (-11/54)*k1[:] + (5/2)*k2[:] + (-70/27)*k3[:] + (35/27)*k4[:])
    k5[:] = h*y_derivs[:]
    
    #k6 values
    y_derivs = dydx(xi + (7/8)*h, yi[:] + (1631/55296)*k1[:] + (175/512)*k2[:] + (575/13824)*k3[:] + (44275/110592)*k4[:] + (253/4096)*k5[:])
    k6[:] = h*y_derivs[:]
    
    #advance y by a step h
    #use values from Cash-Karp value table
    
    yipo = yi + (37/378)*k1 + 0.0*k2 + (250/621)*k3 + (125/594)*k4 + 0.0*k5 + (512/1771)*k6
    
    return yipo

In [ ]:
def cashkarp_core2(dydx,xi,yi,nv,h):
    
    #will give single step
    #xi is x on left side of step
    #nv is number of variables (for this it's 2)
    #yi is an array at location xi
    #h is
    
    #declare k? (multiple integer values of k) arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv) #one element is y, one is z
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
        

   #get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:] #two equations, operating on all elements of array
    
    #k2 values
    y_derivs = dydx(xi + (1/5)*h, yi[:] + (1/5)*k1[:])
    k2[:] = h*y_derivs[:]
    
    #k3 values
    y_derivs = dydx(xi + (3/10)*h, yi[:] + (3/40)*k1[:] + (9/40)*k2[:])
    k3[:] = h*y_derivs[:]
    
    #k4 values
    y_derivs = dydx(xi + (3/5)*h, yi[:] + (3/10)*k1[:] + (-9/10)*k2[:] + (6/5)*k3[:])
    k4[:] = h*y_derivs[:]
    
    #k5 values
    y_derivs = dydx(xi + h, yi[:] + (-11/54)*k1[:] + (5/2)*k2[:] + (-70/27)*k3[:] + (35/27)*k4[:])
    k5[:] = h*y_derivs[:]
    
    #k6 values
    y_derivs = dydx(xi + (7/8)*h, yi[:] + (1631/55296)*k1[:] + (175/512)*k2[:] + (575/13824)*k3[:] + (44275/110592)*k4[:] + (253/4096)*k5[:])
    k6[:] = h*y_derivs[:]
    
    #advance y by a step h
    #use values from Cash-Karp value table
    
    yipo_comp = yi + (2825/27648)*k1 + 0.0*k2 + (18575/48384)*k3 + (13525/55296)*k4 + (277/14336)*k5 + (1/4)*k6
    
    return yipo_comp

Define an adaptative step size driver for Cash-Karp

In [ ]:
def cashkarp_ad(dydx,x_i,y_i,nv,h,tol):
    
    # h = step
    # tolerace
    
    SAFETY = 0.9 # next step can only be 90% smaller
    H_NEW_FAC = 2.0 # next step can only be twice as big
    
    #max iterations and initial iteration
    imax = 10000
    i = 0
    
    #create an error
    delta = np.full(nv,2*tol)
    
    #remember the step
    h_step = h
    
    #adjust step
    while (delta.max()/tol > 1.0):
        
        #estimate our error by taking the difference between yn+1 and yn+1* in Cash-Karp table
        ck1 = cashkarp_core1(dydx,x_i,y_i,nv,h_step)
        ck2 = cashkarp_core2(dydx,x_i,y_i,nv,h_step)

        delta = np.fabs(ck1 - ck2)
        
        #if the error is too large, take a smaller step
        if(delta.max()/tol > 1.0):
            
            h_step *= SAFETY * (delta.max()/tol)**(-0.25)
            
        #check the iteration
        if(i > imax):
            print('Too many iterations in rk4_mv_ad()')
            raise StopIteration('Ending after i = ', i)
        #iterate
        i += 1
        
    #next time, try to take a bigger step
    h_new = np.fmin(h_step * (delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return ck1, h_new, h_step

Define wrapper for Cash-Karp

In [ ]:
def cashkarp(dfdx,a,b,y_a,tol):
    
    #dfdx is the derivative wrtx
    #a is lower bound
    #b is upper bound
    #y_a boundary conditions
    #tol is the tolerance for integrating y
    
    #define starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small
    h = 1.0e-4 * (b-a)
    
    #max iterations
    imax = 10000
    i = 0
    
    #set number of coupled odes to the size of y_a
    nv = len(y_a)
    
    #set the initial condtions
    x = np.full(1,a)
    y = np.full((1,nv), y_a)
    
    #set a flag
    flag = 1
    
    #loop until we reach the right side (we don't know how many steps to take)
    while (flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = cashkarp_ad(dydx,xi,yi,nv,h,tol)
        
        #update step
        h = h_new
        
        #prevent overshoot
        if(xi+h_step>b):
            
            #take smaller step
            h = b - xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = cashkarp_ad(dydx,xi,yi,nv,h,tol)
            
            #break
            flag = 0
            
        #update values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x), nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new
        
        #prevent too many iterations
        if (i>imax):
            print('Max iterations reached')
            raise StopIteration('Iteration number = ', i)
            
        i += 1
        
        #output some information
        s = 'i = %3d\tx = %9.8f\th = %9.8f\tb = %9.8f' % (i,xi,h_step,b)
        print(s)
        
        # break if new xi is == b
        if (xi == b):
            flag = 0
            
    return x,y

Perform the integration

In [ ]:
a = 0.0
b = 2.0*np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

#perform integration
x,y = cashkarp(dydx,a,b,y_0,tolerance)

In [ ]:
plt.plot(x,y[:,0], 'o',label = 'y(x)')
plt.plot(x,y[:,1], 'o',label = 'dydx(x)')
xx = np.linspace(0,2.0*np.pi, 1000)
plt.plot(xx, np.sin(xx), label='sin(x)')
plt.plot(xx, np.cos(xx), label='cos(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)
plt.title('Runge-Kutta Method')

Plot Error

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0]-sine)
dydx_error = (y[:,1]-cosine)

plt.plot(x,y_error,label='y(x) error')
plt.plot(x,dydx_error,label='dydx(x) error')
plt.legend(frameon = False)